## Import Statements and Functions

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import glob
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
# from rnn_utils import *
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 50000)
pd.set_option('display.max_columns', 50000)
pd.set_option('display.width', 100000)
import os
import matplotlib.pyplot as plt
from datetime import datetime

In [62]:
import math
import itertools
from sklearn import preprocessing
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import math

In [63]:
def read_file(filename):
    return pd.read_csv(filename)

def get_data(timeframe):
    return read_file(r"C:\Users\ayush\Desktop\IITB\ZeltaLabPS\BackTesting\data\data\btcusdt_" + timeframe + "_total.csv")

time_frame = "1h"
# Load data
df = get_data(time_frame)
df.head()

,datetime,open,high,low,close,volume
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329


In [64]:
df['change_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4

df['ret_5'] = df['close'].rolling(window=5).mean().pct_change() * 1e4
df['ret_1'] = df['close'].rolling(window=1).mean().pct_change() * 1e4
df['ret_10'] = df['close'].rolling(window=10).mean().pct_change() * 1e4
df['ret_30'] = df['close'].rolling(window=30).mean().pct_change() * 1e4

df['ret_5'] = df['ret_5'].shift(-5)
df['ret_1'] = df['ret_1'].shift(-1)
df['ret_10'] = df['ret_10'].shift(-10)
df['ret_30'] = df['ret_30'].shift(-30)

df = df.dropna()
df = df.reset_index(drop=True)

In [65]:
df.head()

,datetime,open,high,low,close,volume,change_5,ret_5,ret_1,ret_10,ret_30
0,2018-01-01 10:30:00,13615.20,13699.00,13526.50,13558.99,404.229046,4.469738,1.619951,163.301249,-23.245000,-1.720235
1,2018-01-01 11:30:00,13539.00,13800.00,13510.00,13780.41,264.989684,86.039026,-82.285202,-152.433781,-56.936158,-7.266432
2,2018-01-01 12:30:00,13780.00,13818.55,13555.02,13570.35,292.188777,35.485786,-58.971900,-51.848331,-41.184822,7.353075
3,2018-01-01 13:30:00,13569.98,13735.24,13400.00,13499.99,271.813553,13.244840,-72.002119,86.666731,-27.526604,4.100162
4,2018-01-01 14:30:00,13499.97,13670.00,13459.11,13616.99,213.095829,2.349629,-60.903663,-34.501017,-28.482110,1.820772


In [66]:
def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300', 'tick_10'], aggfunc = ['mean', 'median', 'count'], buckets = 10):
# def print_buckets(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300'], aggfunc = ['mean', 'median', 'count']):
# def print_buckets(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60'], aggfunc = ['mean', 'median', 'count']):
    for alpha in alphas:
        msg = df.pivot_table(index = pd.qcut(df[alpha],buckets,duplicates='drop'), values=rets, aggfunc=aggfunc)
        print(msg)
        
def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'ret_60', 'ret_300']):
# def print_corrs(df, alphas, rets = ['ret_10', 'ret_30', 'iv_10']):
# def print_corrs(df, alphas, rets = ['tick_10', 'tick_30', 'tick_60', 'tick_300']):
# def print_corrs(df, alphas, rets = ['ret_60', 'ret_300', 'iv_60']):
    msg = "                                      "
    for ret in rets:
        msg += f"{ret:>8s}"
    print(msg)
    for alpha in alphas:
        msg = f"{alpha:30s} corr -> "
        for ret in rets:
            msg += f"{df[alpha].corr(df[ret])*100:7.2f} "
        print(msg)
    print()

In [73]:
def calculate_momentum(df, n=10):
    df['Momentum'] = df['close'] - df['close'].shift(n)
    df['Momentum'] = df['Momentum'] / df['close'].shift(n) * 100

    #printing the nan count of the momentum column
    print(df['Momentum'].isna().sum())
    df = df.dropna()
    return df

def calculate_roc(df, n=10):
    df['ROC'] = (df['close'] - df['close'].shift(n)) / df['close'].shift(n)
    #printing the nan count of the momentum column
    print(df['ROC'].isna().sum())
    df = df.dropna()
    return df

def calculate_cci(df, n=20, factor=0.0015):
    typical_price = (df['high'] + df['low'] + df['close']) / 3
    sma = typical_price.rolling(window=n).mean()
    mean_deviation = (typical_price - sma).abs().rolling(window=n).mean()
    
    df['CCI'] = (typical_price - sma) / (factor * mean_deviation)

    #printing the nan count of the momentum column
    print(df['CCI'].isna().sum())
    df = df.dropna()
    
    return df

def calculate_alpha(btc_data):
    # Ensure that the data is sorted in ascending order by date
    # btc_data = btc_data.sort_index()

    # Calculate the 5-day delta of close prices
    btc_data['Close_5D_Delta'] = btc_data['close'].diff(5)

    # Calculate the 20-day standard deviation of close prices
    btc_data['Close_20D_STD'] = btc_data['close'].rolling(window=20).std()

    # Calculate the 10-day correlation between close prices and volume
    btc_data['Close_Volume_10D_Corr'] = btc_data['close'].rolling(window=10).corr(btc_data['volume'])

    # Combine the calculated values into the alpha expression
    btc_data['Alpha'] = btc_data['Close_5D_Delta'] / btc_data['Close_20D_STD'] + btc_data['Close_Volume_10D_Corr']
    #printing the nan count of the momentum column
    print(btc_data['Alpha'].isna().sum())
    btc_data = btc_data.dropna()

    return btc_data

def calculate_volatility(df, window=20):
    """
    Calculate historical volatility using daily returns.
    
    Parameters:
    - df: DataFrame with 'Close' prices
    - window: Rolling window size for volatility calculation
    
    Returns:
    - DataFrame with 'Volatility' column
    """
    # Calculate daily returns
    df['Daily_Return'] = df['close'].pct_change()

    # Calculate historical volatility using the rolling standard deviation
    df['Volatility'] = df['Daily_Return'].rolling(window=window).std() * (252**0.5)  # 252 trading days in a year
    #printing the nan count of the momentum column
    print(df['Volatility'].isna().sum())
    df = df.dropna()
    return df.dropna()

In [75]:
df = calculate_volatility(df)
df = calculate_momentum(df)
df = calculate_roc(df)
print(df.head())

df = calculate_cci(df)
df = calculate_alpha(df)
df = df.reset_index(drop=True)

20
10
10
                datetime      open      high       low     close      volume   change_5      ret_5       ret_1     ret_10     ret_30  Daily_Return  Volatility  Momentum       ROC          CCI  Close_5D_Delta  Close_20D_STD  Close_Volume_10D_Corr     Alpha
213  2018-01-10 08:30:00  14211.00  14300.01  13950.00  13964.05  602.167204 -40.971916 -80.812007  198.244779  -7.342817 -12.839894     -0.018482    0.302079 -6.594858 -0.065949 -1050.815483         -290.87     348.607478              -0.697550 -1.531926
214  2018-01-10 09:30:00  13980.00  14323.00  13866.66  14240.88  759.187875 -22.505836 -69.529590  -73.141547 -43.961178 -23.540283      0.019824    0.310643 -3.800198 -0.038002  -773.972477         -159.12     318.259553              -0.631643 -1.131612
215  2018-01-10 10:30:00  14240.88  14255.32  13924.05  14136.72  730.701129  20.934886 -63.973268   58.634535  19.026645 -17.976521     -0.007314    0.300243 -3.948275 -0.039483  -734.197770          147.68     318.127110 

In [76]:
df.head()

,datetime,open,high,low,close,volume,change_5,ret_5,ret_1,ret_10,ret_30,Daily_Return,Volatility,Momentum,ROC,CCI,Close_5D_Delta,Close_20D_STD,Close_Volume_10D_Corr,Alpha
0,2018-01-12 17:30:00,13900.02,14059.48,13738.56,14039.63,498.458511,46.235058,-75.909916,-162.319092,-28.723744,5.238112,0.010044,0.305748,4.775626,0.047756,1225.879720,317.59,297.902155,0.088157,1.154245
1,2018-01-12 18:30:00,14013.55,14047.99,13741.35,13811.74,568.811595,23.294438,-59.832861,8.876507,-5.262554,6.613901,-0.016232,0.301306,-0.276245,-0.002762,911.218885,160.75,302.541841,-0.077024,0.454308
2,2018-01-12 19:30:00,13811.74,13872.99,13601.00,13824.00,591.419100,50.307480,-15.845722,-298.184317,17.324263,7.595244,0.000888,0.301306,0.388293,0.003883,597.967258,347.97,307.237977,-0.088751,1.043824
3,2018-01-12 20:30:00,13824.00,13850.00,13358.99,13411.79,983.004434,-76.158777,72.411220,-34.864847,55.587834,18.043691,-0.029818,0.314523,-1.020000,-0.010200,-8.380300,-529.43,307.734680,-0.453115,-2.173526
4,2018-01-12 21:30:00,13411.79,13600.00,13222.00,13365.03,1128.940060,-77.549191,49.446819,115.959336,50.863724,22.916470,-0.003486,0.314678,-3.578169,-0.035782,-415.095168,-534.99,308.899310,-0.656803,-2.388726


## LSTM 

In [96]:
# Build model
#####################
input_dim = 6
hidden_dim = 64
num_layers = 4
output_dim = 6


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states!
        out = self.fc(out[:, -1, :])
        # out.size() --> 100, 10
        return out

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)
loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())


LSTM(
  (lstm): LSTM(6, 64, num_layers=4, batch_first=True)
  (fc): Linear(in_features=64, out_features=6, bias=True)
)
18
torch.Size([256, 6])
torch.Size([256, 64])
torch.Size([256])
torch.Size([256])
torch.Size([256, 64])
torch.Size([256, 64])
torch.Size([256])
torch.Size([256])
torch.Size([256, 64])
torch.Size([256, 64])
torch.Size([256])
torch.Size([256])
torch.Size([256, 64])
torch.Size([256, 64])
torch.Size([256])
torch.Size([256])
torch.Size([6, 64])
torch.Size([6])


In [87]:
class dataset(Dataset): 
    def __init__(self, dataframe, selected_columns, lookback, fut): 
        if not selected_columns:
            raise ValueError("selected_columns must be a non-empty list of column names")

        self.data = dataframe[selected_columns].values
        self.fut = fut
        # self.data = dataframe
        self.lookback = lookback

        # print(self.data.head())
  
    def __len__(self): 
        return len(self.data) - self.lookback - self.fut
  
    def __getitem__(self, idx):

        x = self.data[idx: idx + self.lookback]  # Convert to NumPy array
        y = self.data[idx + self.lookback: idx + self.lookback + self.fut] # Convert to NumPy array

        return torch.from_numpy(x).float(), torch.from_numpy(y).float()


In [88]:
data = dataset(df, ['close', 'volume', 'CCI', 'Volatility', 'Momentum', 'ROC'], 60, 1)

In [89]:
check = DataLoader(data, shuffle=True, batch_size=8)

In [94]:
import matplotlib.pyplot as plt

## Check a batch of the dataset
# Get a batch using the data loader
dataiter = iter(check)
batch = next(dataiter)

print(batch[0].shape, batch[1].shape, )


torch.Size([8, 60, 6]) torch.Size([8, 1, 6])


In [95]:
train_dataset = df.copy()
train_data = dataset(train_dataset, ['close', 'volume', 'CCI', 'Volatility', 'Momentum', 'ROC'], 60, 1)
train_loader = DataLoader(train_data, shuffle=True, batch_size=8)

In [ ]:
# Train model
#####################
num_epochs = 40
hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim =lookback-1

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()

    # Forward pass
    y_train_pred_ts = model(x_train_ts)

    loss = loss_fn(y_train_pred_ts, y_train_ts)
    if t % 10 == 0 and t !=0:
        print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()